In [12]:
import numpy as np
import tensorflow as tf
import keras
from keras import Model, activations, layers, losses, optimizers, callbacks, regularizers
from keras.layers import Dense, Convolution1D, Dropout, BatchNormalization, concatenate, TimeDistributed, Layer, Input
import matplotlib.pyplot as plt
# import ss_functions
from ss_functions import *
from ss_pred_classes import *

In [7]:
x_data, y_data = get_data('list.txt')
x_train, y_train = x_data[101:], y_data[101:]
fff, y_data = get_data('list.txt', encode_y=False)
x_test, y_test = x_data[:101], y_data[:101]

In [8]:
x_data1, y_data1 = get_data2('list.txt')
x_train1, x_train2 = np.array(x_data1[0][101:]), np.array(x_data1[1][101:])
x_test1, x_test2 = np.array(x_data1[0][:101]), np.array(x_data1[1][:101])

NameError: name 'get_data2' is not defined

In [18]:
block1 = DeepInception_block()
block2 = DeepInception_block()
block3 = DeepInception_block()

inputs = layers.Input((800,41))
X = block1(inputs)
X = block2(X)
X1 = Convolution1D(100, 11, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X)
X1 = TimeDistributed(Dense(256, activation='relu'))(X1)
X1 = Dropout(0.5)(X1)

Y2 = TimeDistributed(Dense(3, activation='softmax'))(X1)
X = block3(X)

# input1 = layers.Input((800,21))
# input2 = layers.Input((800,20))
# X1 = block1(input1)
# X2 = block2(input2)

# X = layers.concatenate([X1,X2])
# X = DeepInception_block()(X)

X = Convolution1D(100, 11, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X)
X = TimeDistributed(Dense(256, activation='relu'))(X)
X = Dropout(0.5)(X)

Y1 = TimeDistributed(Dense(3, activation='softmax'))(X)

model = Model(inputs=inputs, outputs=[Y1,Y2])

opt = optimizers.Adam(learning_rate=0.005)
model.compile(loss='categorical_crossentropy', # try siome: "categorical_focal_crossentropy, adam, sparse_categorical_crossentropy
              optimizer=opt,
              metrics=[truncated_accuracy])

stop_monitor_loss = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=8,
    mode='min',
    verbose=1,
    restore_best_weights=True
)

checkpoint = callbacks.ModelCheckpoint(
    './ss_pred_modeltest9.keras',
    monitor='val_truncated_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)


TypeError: Exception encountered when calling layer "deep_inception_block_4" (type DeepInception_block).

in user code:


    TypeError: tf____init__() takes 1 positional argument but 2 were given


Call arguments received by layer "deep_inception_block_4" (type DeepInception_block):
  • inputs=tf.Tensor(shape=(None, 800, 41), dtype=float32)

In [16]:
def get_model0():
    inputs = Input((800, 41))
    block1 = DeepInception_block()
    X = block1(inputs)
    X = DeepInception_block()(X)
    X = DeepInception_block()(X)

    X = Convolution1D(100, 11, activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.001))(X)
    X = TimeDistributed(Dense(256, activation='relu'))(X)
    X = Dropout(0.4)(X)
    Y = TimeDistributed(Dense(3, activation='softmax'))(X)
    model = Model(inputs=inputs, outputs=Y)
    return model

In [17]:
model0 = get_model0()

opt = optimizers.Adam(learning_rate=0.005)
model.compile(loss='categorical_crossentropy', # try siome: "categorical_focal_crossentropy, adam, sparse_categorical_crossentropy
              optimizer=opt,
              metrics=[truncated_accuracy])

stop_monitor_loss = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=8,
    mode='min',
    verbose=1,
    restore_best_weights=True
)

checkpoint = callbacks.ModelCheckpoint(
    './ss_pred_modeltest9.keras',
    monitor='val_truncated_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

TypeError: Exception encountered when calling layer "deep_inception_block_3" (type DeepInception_block).

in user code:


    TypeError: tf____init__() takes 1 positional argument but 2 were given


Call arguments received by layer "deep_inception_block_3" (type DeepInception_block):
  • inputs=tf.Tensor(shape=(None, 800, 41), dtype=float32)

In [ ]:
with tf.device('/GPU:1'):
    history = model.fit(x_train, (y_train,y_train),
                        epochs=60,
                        batch_size=12,
                        validation_split=0.1,
                        callbacks=[stop_monitor_loss, checkpoint])
    
# 3rd block - negligeable improvement

In [5]:
test_model = keras.models.load_model('ss_pred_modeltest9.keras', 
                                     custom_objects={
                                         'inception_conv': inception_conv,
                                         'InceptionNet_paper': InceptionNet_paper,
                                         'DeepInception_block': DeepInception_block,
                                         'truncated_accuracy': truncated_accuracy
                                     })

TypeError: __init__() got an unexpected keyword argument 'name'

In [ ]:
ss_map = {'C': 0, 'H': 1, 'E': 2}
from_aa = {0: 'C', 1: 'H', 2: 'E'}
_,predictions_hot = model.predict(x_test)
predictions = []
for prediction in predictions_hot:
    dssp = ''
    for i in prediction:
        dssp += from_aa[np.argmax(i)]
    predictions.append(dssp)

total = 0
TP = 0
for prediction, truth in zip(predictions, y_test):
    for i, ss in enumerate(truth):
        total +=1
        if ss==prediction[i]:
            TP+=1

accuracy = TP/total
print(accuracy)

input:
1. one hot encoded sequence
2. PSSM

Model:
1D convolutional neural network

output:
multiclass classification - dense layer with relu activaiton - 3?

validation metric - accuray + model specific measures

soruces:
https://www.csbj.org/article/S2001-0370(22)00506-2/fulltext
